In [11]:
import io
from PIL import Image
import io
import sys
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision.models import resnet34
import torch.nn.functional as F

In [12]:
TRANSFORM = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4
MODEL = resnet34(pretrained=False) ## change it to True before training
NUM_CLASSES = 10
LEARNING_RATE = 10e-3

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
class ImageClassifier:
    def __init__(self):
        # Class module from AI team        
        self.classifier = Classifier()
        # Model path with pth file
        model_path = 'drive/MyDrive/model/ResNet34.pth'               
        self.classifier.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

    def predict(self, image): 
        # Preprocessing image      
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        preprocess = transforms.Compose([transforms.RandomResizedCrop(224),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         normalize])
        image = preprocess(image) 
        image = image.unsqueeze(0)
        # Predict image from classifier
        preds = F.softmax(self.classifier(image), dim=1)
        # Get a results with tensor value
        results = torch.topk(preds.cpu().data, k=3, dim=1)
        # Get tensor classes and convert it to list 
        classes = results[1][0]
        classes = classes.tolist()
        # Get tensor values and convert it to list
        value = results[0][0]
        v = value.tolist()
        # To dictionary
        res = {classes[i]: v[i] for i in range(len(classes))}
        # Get key of max value from dictionary
        max_value = max(res, key=res.get)
        # Open class txt for know what the predicted value from the model
        with open('class.txt', 'r') as f:
          to_label = eval(f.read())
        # Get result final and return it
        result_final = to_label[max_value]
        return result_final

### PREDICTION

In [15]:
# Call Image Classifier class for BE side
image_classifier = ImageClassifier()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.core.module.LightningModule class instead.
  "pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7"


In [17]:
# Testing image
image = IMAGE
image = Image.open(image)
predicted_class = image_classifier.predict(image)

print(predicted_class)

Coat
